In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bipolar_disorder/GSE46416'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"State- and trait-specific gene expression in euthymia and mania"
!Series_summary	"Gene expression profiles of bipolar disorder (BD) patients were assessed during both a manic and a euthymic phase and compared both intra-individually, and with the gene expression profiles of controls."
!Series_overall_design	"11 BD patients were assessed in their manic as well as in their euthymic phase. Comparison of gene expression BD euthymic vs. controls, BD manic vs. controls, and intra-patient comparison BD euthymic vs. BD manic."
Sample Characteristics Dictionary:
{0: ['tissue: blood'], 1: ['disease status: bipolar disorder (BD)', 'disease status: control'], 2: ['bd phase: mania', 'bd phase: euthymia', nan], 3: ['patient identifier: 103623', 'patient identifier: 103625', 'patient identifier: 103626', 'patient identifier: 103627', nan, 'patient identifier: 103620', 'patient identifier: 101701', 'patient identifier: 102391', 'patient identifier: 102394', 'pati

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check for gene expression data availability
# Based on series summary and design, it looks like gene expression data is available
is_gene_available = True

# Variable availability analysis
# 'Bipolar_disorder' -> likely in "disease status"
if len(set(['bipolar disorder (BD)', 'control'])) > 1:
    trait_row = 1  # referring to 'disease status: bipolar disorder (BD)', 'disease status: control'

# Assuming age and gender information might be missing in given sample characteristics
# Otherwise, sample characteristics dictionary should include relevant keys if available
age_row = None  # since age data is not explicitly mentioned
gender_row = None  # since gender data is not explicitly mentioned

# Defining conversion functions
def convert_trait(value):
    # Extract the value after the colon
    val = value.split(":")[1].strip().lower()
    if val == "bipolar disorder (BD)".lower():
        return 1
    elif val == "control".lower():
        return 0
    return None

# As the age and gender rows are not found, we skip their conversion functions
convert_age = None
convert_gender = None

# Save cohort information
save_cohort_info('GSE46416', './preprocessed/Bipolar_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bipolar_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bipolar_disorder/trait_data/GSE46416.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1129903': [1], 'GSM1129904': [1], 'GSM1129905': [1], 'GSM1129906': [1], 'GSM1129907': [1], 'GSM1129908': [1], 'GSM1129909': [1], 'GSM1129910': [1], 'GSM1129911': [0], 'GSM1129912': [0], 'GSM1129913': [0], 'GSM1129914': [0], 'GSM1129915': [1], 'GSM1129916': [1], 'GSM1129917': [0], 'GSM1129918': [1], 'GSM1129919': [1], 'GSM1129920': [0], 'GSM1129921': [0], 'GSM1129922': [1], 'GSM1129923': [1], 'GSM1129924': [1], 'GSM1129925': [0], 'GSM1129926': [0], 'GSM1129927': [0], 'GSM1129928': [1], 'GSM1129929': [1], 'GSM1129930': [1], 'GSM1129931': [1], 'GSM1129932': [1], 'GSM1129933': [1], 'GSM1129934': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2315252', '2315253', '2315374', '2315375', '2315376', '2315377',
       '2315586', '2315587', '2315588', '2315589', '2315591', '2315594',
       '2315595', '2315596', '2315598', '2315602', '2315603', '2315604',
       '2315605', '2315606'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315252', '2315253', '2315374', '2315375', '2315376'], 'transcript_cluster_id': ['2315251', '2315251', '2315373', '2315373', '2315373'], 'gene_symbol': [nan, nan, nan, nan, nan], 'cytoband': [nan, nan, nan, nan, nan], 'mRNA_accession': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'chromosome': [nan, nan, nan, nan, nan], 'RANGE_GB': [nan, nan, nan, nan, nan], 'RANGE_STRAND': [nan, nan, nan, nan, nan], 'RANGE_START': [nan, nan, nan, nan, nan], 'RANGE_STOP': [nan, nan, nan, nan, nan], 'SPOT_ID': ['2315252', '2315253', '2315374', '2315375', '2315376']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant keys in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'gene_symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert gene probes into gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Bipolar_disorder/gene_data/GSE46416.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Bipolar_disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE46416', './preprocessed/Bipolar_disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Bipolar_disorder/GSE46416.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Bipolar_disorder', the least common label is '0.0' with 10 occurrences. This represents 31.25% of the dataset.
The distribution of the feature 'Bipolar_disorder' in this dataset is fine.

